# **MCS14 Music Separation**

In [1]:
# @title 1. Install Dependencies
%%capture
%pip install demucs streamlit pyngrok flask flask-ngrok
!npm install localtunnel
!git clone https://github.com/Lim1802/MCS14_FIT3162_SINGING_VIDEO_SEPERATION.git
%cd /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs
! pip install streamlit-js-eval
! pip install streamlit-extras
! pip install streamlit-extras

In [ ]:
# @title 2. Streamlit Code
%%writefile app.py
import streamlit as st
import subprocess
import os
import soundfile as sf
import os
from streamlit_js_eval import streamlit_js_eval
from streamlit_extras.stylable_container import stylable_container

# Create the .streamlit directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.streamlit"), exist_ok=True)

# Write the config.toml file
with open(os.path.expanduser("~/.streamlit/config.toml"), "w") as f:
    f.write("""[theme]
primaryColor = "#1f77b4"  # Blue color
backgroundColor = "#ffffff"  # White background
secondaryBackgroundColor = "#f0f8ff"  # Lighter white for sidebar and other secondary sections
textColor = "#000000"  # Black text color for readability
font = "sans serif"

[server]
maxUploadSize = 50  # max upload size
""")


st.title("Music Separation :notes:")

if 'seperated_audio' not in st.session_state:
    st.session_state.seperated_audio = ''

if 'uploaded_file' not in st.session_state:
    st.session_state.uploaded_file = ''

def del_files():
  streamlit_js_eval(js_expressions="parent.window.location.reload()")


# File uploader
st.session_state.uploaded_file = st.file_uploader("Choose a WAV or MP3 file", type=["wav", "mp3"])

if st.session_state.uploaded_file:
    uploaded_file = st.session_state.uploaded_file
    # Display the uploaded audio
    if uploaded_file.type == "audio/mp3":
      st.audio(uploaded_file, format="audio/mp3")

      # Save the uploaded file temporarily
      with open("temp_input.mp3", "wb") as f:
          f.write(uploaded_file.getvalue())

    else:
      st.audio(uploaded_file, format="audio/wav")

      # Save the uploaded file temporarily
      with open("temp_input.wav", "wb") as f:
          f.write(uploaded_file.getvalue())

    # Button to start processing
    if st.button("Separate Audio Sources"):
        # Run separation model (20 epochs)
        with st.spinner('Processing audio...'):
            command = f"python -m demucs.separate -n d7014c1b --repo /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs/release_models temp_input.wav"
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()

        if process.returncode == 0:
            st.success("Audio separation completed successfully!")

            # Find the output directory
            output_dir = "/content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs/separated/d7014c1b/temp_input"
            st.session_state.seperated_audio = output_dir

        else:
            st.error("Error during audio separation. Please check your input file and try again.")
            st.text(stderr.decode())

    if st.session_state.seperated_audio:
      if os.path.exists(st.session_state.seperated_audio):
        output_dir = st.session_state.seperated_audio
        for file in os.listdir(output_dir):
            if file.endswith(".wav"):
                file_path = os.path.join(output_dir, file)
                st.write(f"Separated {file}:")
                st.audio(file_path, format="audio/wav")

                # Provide download button for each separated file
                with open(file_path, "rb") as f:
                    st.download_button(
                        label=f"Download {file}",
                        data=f,
                        file_name=file,
                        mime="audio/wav"
                    )
        restart = st.button("Restart", key="restart_button", on_click=del_files)


    # Clean up temporary file
    if os.path.exists("temp_input.wav"):
        os.remove("temp_input.wav")

In [ ]:
# @title 3. Web App - Deployment Code
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

!npx localtunnel --port 8501